In [1]:
from seeq import spy
import pandas as pd

# Set the compatibility option so that you maximize the chance that SPy will remain compatible with your notebook/script
spy.options.compatibility = 190

In [2]:
# Log into Seeq Server if you're not using Seeq Data Lab:
spy.login(url='http://localhost:34216', credentials_file='../credentials.key', force=False)

# spy.swap

Performs "asset swapping" functionality equivalent to what you can achieve in Seeq Workbench by the "Swap in this asset..." button for an Asset found in the Data tab.

When you create a calculated item (Signal, Condition, Scalar, Threshold Metric) that is based on stored signals that are part of an asset tree, Seeq allows you to "swap" the original asset that you used for a different asset that also has all of the constituent signals (named identically).

You can then pull the data for the "swapped" calculation just like you would pull data for any other calculated item.

Let's walk through an example.

## Example

First we will query for some signals to use in our calculation and then push a calculation based on those signals.

In [3]:
area_a_df = spy.search({
    'Path': 'Example >> Cooling Tower 1',
    'Asset': 'Area A',
    'Name': 'Temperature'
})

pushed_items_df = spy.push(metadata=pd.DataFrame([{
            'Name': f'Added',
            'Formula': '$t + 10',
            'Formula Parameters': {'$t': area_a_df.iloc[0]}
        }, {
            'Name': f'Hot',
            'Formula': '$t > 80',
            'Formula Parameters': {'$t': area_a_df.iloc[0]}
        }, {
            'Name': f'Average on One Day in 2016',
            'Formula': "$t.average(capsule('2016-12-18'))",
            'Formula Parameters': {'$t': area_a_df.iloc[0]}
        }]), workbook='SPy Documentation Examples >> spy.swap')

pushed_items_df

,Name,Formula,Formula Parameters,Type,Scoped To,Datasource Class,Datasource ID,Data ID,ID,Push Result
0,Added,$t + 10,[t=9B90B02D-4B58-405F-9160-6982620654C2],CalculatedSignal,4DA4FE62-79F9-4163-AC85-831ABD16CFC0,Seeq Data Lab,Seeq Data Lab,[4DA4FE62-79F9-4163-AC85-831ABD16CFC0] {Signal...,01E6E5DA-2FA4-4A45-80C2-B8629EF47B38,Success
1,Hot,$t > 80,[t=9B90B02D-4B58-405F-9160-6982620654C2],CalculatedCondition,4DA4FE62-79F9-4163-AC85-831ABD16CFC0,Seeq Data Lab,Seeq Data Lab,[4DA4FE62-79F9-4163-AC85-831ABD16CFC0] {Condit...,36B31F55-BC5D-4A63-A3A1-C42843681664,Success
2,Average on One Day in 2016,$t.average(capsule('2016-12-18')),[t=9B90B02D-4B58-405F-9160-6982620654C2],CalculatedScalar,4DA4FE62-79F9-4163-AC85-831ABD16CFC0,Seeq Data Lab,Seeq Data Lab,[4DA4FE62-79F9-4163-AC85-831ABD16CFC0] {Scalar...,16A60552-8881-4DCC-B0AD-62AE41E329C4,Success


Now we'll query for the assets to swap in for the original Area A asset that we built the calculation for:

In [4]:
areas_def_df = spy.search({
    'Path': 'Example >> Cooling Tower 2',
    'Name': '/Area [DEF]/',
    'Type': 'Asset'
}, old_asset_format=False)

areas_def_df

,Path,Name,Type,Time,Count,Pages,Result
0,Example >> Cooling Tower 2,/Area [DEF]/,Asset,00:00:00.04,3,1,Success


,ID,Path,Asset,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived
0,A30EB497-1BBC-44E7-AFF6-64DBCB1C6B11,Example >> Cooling Tower 2,Area D,Area D,NaN,Asset,NaN,Example Data,False
1,D1CABA52-0B72-416F-8731-797EF7A4E462,Example >> Cooling Tower 2,Area F,Area F,NaN,Asset,NaN,Example Data,False
2,0F64683B-51C6-4D90-85CD-9747B3CA6932,Example >> Cooling Tower 2,Area E,Area E,NaN,Asset,NaN,Example Data,False


Now all we have to do is execute the `spy.swap()` command to swap in these assets and get a bunch of new IDs we can use in `spy.pull()`:

In [5]:
swapped_df = spy.swap(pushed_items_df, areas_def_df, errors='catalog')
swapped_df

,Original ID,ID,Type,Name,Swap Result,Swap Performed
0,01E6E5DA-2FA4-4A45-80C2-B8629EF47B38,00230FE9-6622-45C7-922F-CA36D7E1736A,CalculatedSignal,Added,Success,Example >> Cooling Tower 1 >> Area A --> Examp...
1,36B31F55-BC5D-4A63-A3A1-C42843681664,5B0D8F35-B839-4544-B17E-7935A26DC143,CalculatedCondition,Hot,Success,Example >> Cooling Tower 1 >> Area A --> Examp...
2,16A60552-8881-4DCC-B0AD-62AE41E329C4,753E9810-30B8-4E66-888F-5DAB6183B8C4,CalculatedScalar,Average on One Day in 2016,Success,Example >> Cooling Tower 1 >> Area A --> Examp...
3,01E6E5DA-2FA4-4A45-80C2-B8629EF47B38,NaN,CalculatedSignal,Added,"(400) Bad Request - Temperature not swapped, u...",Example >> Cooling Tower 1 >> Area A --> Examp...
4,36B31F55-BC5D-4A63-A3A1-C42843681664,NaN,CalculatedCondition,Hot,"(400) Bad Request - Temperature not swapped, u...",Example >> Cooling Tower 1 >> Area A --> Examp...
5,16A60552-8881-4DCC-B0AD-62AE41E329C4,NaN,CalculatedScalar,Average on One Day in 2016,"(400) Bad Request - Temperature not swapped, u...",Example >> Cooling Tower 1 >> Area A --> Examp...
6,01E6E5DA-2FA4-4A45-80C2-B8629EF47B38,9A82A988-B842-42D1-87C9-5C222009ACAC,CalculatedSignal,Added,Success,Example >> Cooling Tower 1 >> Area A --> Examp...
7,36B31F55-BC5D-4A63-A3A1-C42843681664,72F90934-7C01-4027-B582-91543475BD9F,CalculatedCondition,Hot,Success,Example >> Cooling Tower 1 >> Area A --> Examp...
8,16A60552-8881-4DCC-B0AD-62AE41E329C4,1ACA4004-C03E-41F8-ABA0-70A55DEE554C,CalculatedScalar,Average on One Day in 2016,Success,Example >> Cooling Tower 1 >> Area A --> Examp...


You'll notice a few things about the returned DataFrame:

- The `Original ID` column corresponds to the calculated item for which we're swapping assets. The values are repeated three times because we're swapping three assets across three items (a signal, condition and scalar). That's why there's nine rows total.
- Some of the rows don't have an `ID` field because there were errors, which can be seen in the `Swap Result` column.
- The `Swap Performed` column includes precise details about what was swapped. The format is `Swapped Out Asset --> Swapped In Asset`.

In order to get a better look at the `Swap Result` and `Swap Performed` columns, let's remove some of the other columns:

In [6]:
pd.set_option('display.max_colwidth', None)
swapped_df[['ID', 'Name', 'Swap Result', 'Swap Performed']]

,ID,Name,Swap Result,Swap Performed
0,00230FE9-6622-45C7-922F-CA36D7E1736A,Added,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area D
1,5B0D8F35-B839-4544-B17E-7935A26DC143,Hot,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area D
2,753E9810-30B8-4E66-888F-5DAB6183B8C4,Average on One Day in 2016,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area D
3,NaN,Added,"(400) Bad Request - Temperature not swapped, unable to swap out Area A and swap in Area F",Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area F
4,NaN,Hot,"(400) Bad Request - Temperature not swapped, unable to swap out Area A and swap in Area F",Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area F
5,NaN,Average on One Day in 2016,"(400) Bad Request - Temperature not swapped, unable to swap out Area A and swap in Area F",Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area F
6,9A82A988-B842-42D1-87C9-5C222009ACAC,Added,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area E
7,72F90934-7C01-4027-B582-91543475BD9F,Hot,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area E
8,1ACA4004-C03E-41F8-ABA0-70A55DEE554C,Average on One Day in 2016,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area E


In Seeq's Example data, Area F does not include a `Temperature` signal, so `spy.swap()` could not create a swapped calculation for that particular asset.

For the swaps that succeeded, you can now pull the data for these swapped items like so:

In [7]:
# First we'll drop the rows that had errors.
successful_swaps_df = swapped_df[swapped_df['Swap Result'] == 'Success']

# Note that we use header='ID' so that each column is uniquely named.
spy.pull(successful_swaps_df, start='2021-01-01', end='2021-01-02', header='ID')

,ID,Name,Time,Count,Pages,Result
0,00230FE9-6622-45C7-922F-CA36D7E1736A,Added,00:00:01.20,97,1,Success
1,5B0D8F35-B839-4544-B17E-7935A26DC143,Hot,00:00:00.25,2,1,Success
2,753E9810-30B8-4E66-888F-5DAB6183B8C4,Average on One Day in 2016,00:00:00.23,1,0,Success
6,9A82A988-B842-42D1-87C9-5C222009ACAC,Added,00:00:01.20,97,1,Success
7,72F90934-7C01-4027-B582-91543475BD9F,Hot,00:00:00.25,2,1,Success
8,1ACA4004-C03E-41F8-ABA0-70A55DEE554C,Average on One Day in 2016,00:00:00.23,1,0,Success


,00230FE9-6622-45C7-922F-CA36D7E1736A,5B0D8F35-B839-4544-B17E-7935A26DC143,753E9810-30B8-4E66-888F-5DAB6183B8C4,9A82A988-B842-42D1-87C9-5C222009ACAC,72F90934-7C01-4027-B582-91543475BD9F,1ACA4004-C03E-41F8-ABA0-70A55DEE554C
2021-01-01 00:00:00+00:00,96.583716,1.0,75.860678,101.209415,1.0,81.768867
2021-01-01 00:15:00+00:00,96.990286,1.0,75.860678,100.564875,1.0,81.768867
2021-01-01 00:30:00+00:00,97.396855,1.0,75.860678,101.359711,1.0,81.768867
2021-01-01 00:45:00+00:00,97.803424,1.0,75.860678,101.776450,1.0,81.768867
2021-01-01 01:00:00+00:00,98.209993,1.0,75.860678,101.785881,1.0,81.768867
...,...,...,...,...,...,...
2021-01-01 23:00:00+00:00,95.488781,1.0,75.860678,101.164443,1.0,81.768867
2021-01-01 23:15:00+00:00,95.957709,1.0,75.860678,102.506365,1.0,81.768867
2021-01-01 23:30:00+00:00,96.318568,1.0,75.860678,103.861894,1.0,81.768867
2021-01-01 23:45:00+00:00,97.208800,1.0,75.860678,104.494810,1.0,81.768867


## Multi-asset Swaps

Calculations can be based on more than one asset. That means swapping is more complex, because you must specify a pairing for each swappable asset. You accomplish this using "Swap Groups".

In the following example, we'll push a calculation that is a summation of `Compressor Power` for Area A and Area B. Then we'll swap out those assets for Areas D & E and then again for Areas D & F.

In [8]:
areas_df = spy.search({
    'Path': 'Example >> Cooling Tower 1',
    'Asset': '/Area [AB]/',
    'Name': 'Compressor Power'
})

added_together_df = spy.push(metadata=pd.DataFrame([{
    'Name': 'Added Together',
    'Formula': '$a + $b',
    'Formula Parameters': {
        '$a': areas_df[areas_df['Asset'] == 'Area A'],
        '$b': areas_df[areas_df['Asset'] == 'Area B']
    }
}]), workbook='SPy Documentation Examples >> spy.swap')

added_together_df = spy.search({
    'Name': 'Added Together'
}, include_swappable_assets=True, workbook='SPy Documentation Examples >> spy.swap')

added_together_df

,Name,Time,Count,Pages,Result
0,Added Together,00:00:00.01,1,1,Success


,ID,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived,Swappable Assets
0,A57F2073-2EE7-42AC-87A2-12E1F3B15A96,Added Together,NaN,CalculatedSignal,kW,Seeq Data Lab,False,ID Type Path \ 0 B4814537-616E-43D8-B19D-42421DB4CADF Asset Example >> Cooling Tower 1 1 E306E388-1852-4FEC-A254-E2F43DB02040 Asset Example >> Cooling Tower 1 Asset 0 Area A 1 Area B


Notice that we used the `spy.search(include_swappable_assets=True)` flag, which means that the `added_together_df` DataFrame includes a `Swappable Assets` column where each cell is an embedded DataFrame detailing the assets upon which the calculation is based:

In [9]:
swappable_assets_df = added_together_df.iloc[0]['Swappable Assets']
swappable_assets_df

,ID,Type,Path,Asset
0,B4814537-616E-43D8-B19D-42421DB4CADF,Asset,Example >> Cooling Tower 1,Area A
1,E306E388-1852-4FEC-A254-E2F43DB02040,Asset,Example >> Cooling Tower 1,Area B


That's what we need in order to specify our "swap groups". Now we have to create a DataFrame of assets with a `Swap Group` and `Swap Out` columns to achieve the pairing we talk about above:

In [10]:
# Grab a DataFrame that contains Areas D, E and F
cooling_tower_2_df = spy.search({
    'Path': 'Example >> Cooling Tower 2',
    'Type': 'Asset'
}, old_asset_format=False).sort_values(by='Name')

# Create a DataFrame that includes the combination of areas:
swap_groups_df = pd.DataFrame([
    cooling_tower_2_df.iloc[0],  # Area D
    cooling_tower_2_df.iloc[1],  # Area E

    cooling_tower_2_df.iloc[0],  # Area D
    cooling_tower_2_df.iloc[2]]) # Area F
    
# Now we'll specify our "Swap Group" and "Swap Out" columns. The Swap Group column can be any signifier you wish to use
# to group the swaps together. Any rows with the same value for the Swap Group will be grouped together for swapping
# purposes.
swap_groups_df['Swap Group'] = [1, 1, 2, 2]

# The "Swap Out" column can be an asset ID string, the latter part of an asset's path, or a DataFrame row for an asset.
# It must be for an asset that is part of the "Swappable Assets" found above.
swap_groups_df['Swap Out'] = [
    swappable_assets_df.iloc[0],  # Area A
    swappable_assets_df.iloc[1],  # Area B

    swappable_assets_df.iloc[0],  # Area A
    swappable_assets_df.iloc[1],  # Area B
]

swap_groups_df

,Path,Type,Time,Count,Pages,Result
0,Example >> Cooling Tower 2,Asset,00:00:00.04,3,1,Success


,ID,Path,Asset,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived,Swap Group,Swap Out
0,A30EB497-1BBC-44E7-AFF6-64DBCB1C6B11,Example >> Cooling Tower 2,Area D,Area D,NaN,Asset,NaN,Example Data,False,1,"ID B4814537-616E-43D8-B19D-42421DB4CADF Type Asset Path Example >> Cooling Tower 1 Asset Area A Name: 0, dtype: object"
2,0F64683B-51C6-4D90-85CD-9747B3CA6932,Example >> Cooling Tower 2,Area E,Area E,NaN,Asset,NaN,Example Data,False,1,"ID E306E388-1852-4FEC-A254-E2F43DB02040 Type Asset Path Example >> Cooling Tower 1 Asset Area B Name: 1, dtype: object"
0,A30EB497-1BBC-44E7-AFF6-64DBCB1C6B11,Example >> Cooling Tower 2,Area D,Area D,NaN,Asset,NaN,Example Data,False,2,"ID B4814537-616E-43D8-B19D-42421DB4CADF Type Asset Path Example >> Cooling Tower 1 Asset Area A Name: 0, dtype: object"
1,D1CABA52-0B72-416F-8731-797EF7A4E462,Example >> Cooling Tower 2,Area F,Area F,NaN,Asset,NaN,Example Data,False,2,"ID E306E388-1852-4FEC-A254-E2F43DB02040 Type Asset Path Example >> Cooling Tower 1 Asset Area B Name: 1, dtype: object"


Now we swap! Note that we're going to have two new swapped calculations -- one for each Swap Group. You can see that there are a couple of actions noted in the `Swap Performed` column (separated by a `\n` line break).

In [11]:
swapped_df = spy.swap(added_together_df, swap_groups_df)

# Use a specific set of display properties so that the linebreaks are rendered correctly
display(swapped_df[['ID', 'Name', 'Swap Result', 'Swap Performed']].style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap',
}))

,ID,Name,Swap Result,Swap Performed
0,9ED34482-5D85-4B6C-BD93-691FCED27B43,Added Together,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area D >> Area D Example >> Cooling Tower 1 >> Area B --> Example >> Cooling Tower 2 >> Area E >> Area E
1,F6512702-089A-4BD4-B987-2866B442FBC3,Added Together,Success,Example >> Cooling Tower 1 >> Area A --> Example >> Cooling Tower 2 >> Area D >> Area D Example >> Cooling Tower 1 >> Area B --> Example >> Cooling Tower 2 >> Area F >> Area F


We can pull just like we did before:

In [12]:
spy.pull(swapped_df, start='2021-01-01', end='2021-01-02', header='ID')

,ID,Name,Time,Count,Pages,Result
0,9ED34482-5D85-4B6C-BD93-691FCED27B43,Added Together,00:00:00.16,97,1,Success
1,F6512702-089A-4BD4-B987-2866B442FBC3,Added Together,00:00:00.16,97,1,Success


,9ED34482-5D85-4B6C-BD93-691FCED27B43,F6512702-089A-4BD4-B987-2866B442FBC3
2021-01-01 00:00:00+00:00,29.362794,28.016513
2021-01-01 00:15:00+00:00,32.655979,28.082996
2021-01-01 00:30:00+00:00,28.149479,28.149479
2021-01-01 00:45:00+00:00,32.784786,28.215962
2021-01-01 01:00:00+00:00,32.813098,28.282445
...,...,...
2021-01-01 23:00:00+00:00,27.369611,27.335406
2021-01-01 23:15:00+00:00,32.330629,27.660053
2021-01-01 23:30:00+00:00,28.261489,28.236783
2021-01-01 23:45:00+00:00,32.405502,27.776533


## Multi-level Swaps

If you have a hierarchy that contains "commonly named" nodes at the bottom of the tree, you'll need to do a multi-level swap. Here's an example where "Raw" and "Cleansed" are categorical child "assets" that are common between the uniquely-named assets (Area A & Area B):

```
Multi-level Swap Example >> Area A >> Raw      >> Temperature
                                      Cleansed >> Temperature
                            Area B >> Raw      >> Temperature
                                      Cleansed >> Temperature
```

You must specify the uniquely-named assets as your "Swap Out" values, and SPy will figure out how to perform the swap at the lower level. Here's an example where the categories are _Raw_ and _Cleansed_:

In [13]:
# First we have to create the asset tree we want from Example data.

area_ab_df = spy.search({
    'Path': 'Example >> Cooling Tower 1',
    'Asset': '/Area [AB]/',
    'Name': 'Temperature'
}, old_asset_format=False)

raw_signals = area_ab_df.copy()
raw_signals['Path'] = 'Multi-level Swap Example >> ' + raw_signals['Asset']
raw_signals['Asset'] = 'Raw'
raw_signals['Reference'] = True

cleansed_signals = area_ab_df.copy()
cleansed_signals['Path'] = 'Multi-level Swap Example >> ' + cleansed_signals['Asset']
cleansed_signals['Asset'] = 'Cleansed'
cleansed_signals['Reference'] = True

combined_signal = pd.DataFrame([{
    'Name': 'Raw and Cleansed Averaged',
    'Type': 'Signal',
    'Formula': '($r + $c) / 2',
    'Formula Parameters': {
        '$r': {'Path': 'Multi-level Swap Example >> Area A', 'Asset': 'Raw',      'Name': 'Temperature'},
        '$c': {'Path': 'Multi-level Swap Example >> Area A', 'Asset': 'Cleansed', 'Name': 'Temperature'}
    }
}])

pushed_items_df = spy.push(metadata=pd.concat([raw_signals, cleansed_signals, combined_signal], ignore_index=True),
                           workbook='SPy Documentation Examples >> spy.swap')

pushed_items_df[['ID', 'Type', 'Path', 'Asset', 'Name']]

,ID,Type,Path,Asset,Name
0.0,760E550C-40DC-4DA6-A1DD-BFEC52A40236,CalculatedSignal,Multi-level Swap Example >> Area A,Raw,Temperature
1.0,3141E645-5697-4EEB-9AB6-B76F005959D8,CalculatedSignal,Multi-level Swap Example >> Area B,Raw,Temperature
2.0,67AFC418-C662-436C-888F-F2050EE8497E,CalculatedSignal,Multi-level Swap Example >> Area A,Cleansed,Temperature
3.0,CDACB0A6-600A-4BAF-BC1F-C464612E27CF,CalculatedSignal,Multi-level Swap Example >> Area B,Cleansed,Temperature
4.0,BFAD6A4B-6BB7-463F-85D0-6A348C601039,CalculatedSignal,NaN,NaN,Raw and Cleansed Averaged
NaN,E34158ED-E669-4B83-B6E0-518ED05BC35A,Asset,,Multi-level Swap Example,Multi-level Swap Example
NaN,944BC81E-C803-4B3C-A8E3-BE06409E7D32,Asset,Multi-level Swap Example,Area A,Area A
NaN,F03B07AA-E467-40FF-9578-8E910F54A0E2,Asset,Multi-level Swap Example >> Area A,Raw,Raw
NaN,21EE8916-93E3-4220-9639-93415231F050,Asset,Multi-level Swap Example,Area B,Area B
NaN,CEDFF626-9561-415F-A9F5-D32AA246E31D,Asset,Multi-level Swap Example >> Area B,Raw,Raw


In [14]:
raw_plus_cleansed = pushed_items_df[pushed_items_df['Name'] == 'Raw and Cleansed Averaged']
area_a_df = pushed_items_df[(pushed_items_df['Type'] == 'Asset') & (pushed_items_df['Name'] == 'Area A')]
area_b_df = pushed_items_df[(pushed_items_df['Type'] == 'Asset') & (pushed_items_df['Name'] == 'Area B')]

# Create a Swap In / Swap Out pairing that uses the unique level of the tree, namely:
#  Swap Out:  test_multilevel_swap >> Example >> Cooling Tower 1 >> Area A
#  Swap In:   test_multilevel_swap >> Example >> Cooling Tower 1 >> Area B

assets_df = area_b_df.copy()                  # Swap In
assets_df['Swap Out'] = [area_a_df.iloc[0]]   # Swap Out

swap_results = spy.swap(raw_plus_cleansed, assets_df)

display(swap_results[['ID', 'Name', 'Swap Performed']].style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap',
}))

,ID,Name,Swap Performed
0,ACA1ABB3-4D86-41F7-B0C2-5E475D5342F9,Raw and Cleansed Averaged,Multi-level Swap Example >> Area A >> Raw --> Multi-level Swap Example >> Area B >> Raw Multi-level Swap Example >> Area A >> Cleansed --> Multi-level Swap Example >> Area B >> Cleansed


## Detailed Help

All SPy functions have detailed documentation to help you use them. Just execute `help(spy.<func>)` like
you see below.

**Make sure you re-execute the cell below to see the latest documentation. It otherwise might be from an
earlier version of SPy.**

In [15]:
help(spy.swap)

Help on function swap in module seeq.spy._swap:

swap(items: pandas.core.frame.DataFrame, assets: pandas.core.frame.DataFrame, *, old_asset_format: Union[bool, NoneType] = None, errors: Union[str, NoneType] = None, quiet: Union[bool, NoneType] = False, status: Union[seeq.spy._status.Status, NoneType] = None, session: Union[seeq.spy._session.Session, NoneType] = None)
    Operates on a DataFrame of items by swapping out the assets that those
    items are based on. The returned DataFrame can be supplied to
    spy.pull() to retrieve the resulting data.
    
    Parameters
    ----------
    items : {pd.DataFrame}
        A DataFrame of items over which to perform the swapping operation. The
        only required column is ID. Typically, you will have derived this
        DataFrame via a spy.search() or spy.push(metadata) call.
    
    assets : {pd.DataFrame}
        A DataFrame of Asset items (and ONLY Asset items) to swap IN. Each row
        must have valid ID, Type, Path, Asset, and